# Generating Graphs Based upon Mean and SD

Mean and SD are statistically significant so we want to provide the user with some statistical information to guide them through their decision making process.

Standard imports

In [2]:
import pandas as pd
import numpy as np


# Import needed for Data Exploration and Algorithm Developing\

# Import Useful for plotting
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
from sklearn import preprocessing


data = pd.read_csv('../data/course-semesters.csv')
orig = pd.read_csv('../data/registrations.csv')

/home/tj/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
orig.head()

,studentid,semester,grade,major,concentration,code,section,coursetitle,sectiontitle,professor,course-sectiontitle,code-semester
0,30876146,0708FA,SR,Engineering,Computing,AHS CAP SPR,1,AHS CapstoneSpring Pre-registration,NaN,"Lynch, Caitrin",AHS CapstoneSpring Pre-registration,AHS CAP SPR 0708FA
1,23925034,0607FA,SR,Engineering,Bioengineering,AHS CAP SPR,1,AHS CapstoneSpring Pre-registration,NaN,"Martello, Robert",AHS CapstoneSpring Pre-registration,AHS CAP SPR 0607FA
2,99052948,0607FA,SR,Electr'l & Computer Engr,NaN,AHS CAP SPR,1,AHS CapstoneSpring Pre-registration,NaN,"Martello, Robert",AHS CapstoneSpring Pre-registration,AHS CAP SPR 0607FA
3,29826367,0607FA,SR,Mechanical Engineering,NaN,AHS CAP SPR,1,AHS CapstoneSpring Pre-registration,NaN,"Martello, Robert",AHS CapstoneSpring Pre-registration,AHS CAP SPR 0607FA
4,91614816,0607FA,SR,Mechanical Engineering,NaN,AHS CAP SPR,1,AHS CapstoneSpring Pre-registration,NaN,"Martello, Robert",AHS CapstoneSpring Pre-registration,AHS CAP SPR 0607FA


In [4]:
data.head()

,code-semester,code,semester,name,prof,redundant_codes,prereqs,student_ids
0,AHS CAP SPR 0607FA,AHS CAP SPR,0607FA,AHS CapstoneSpring Pre-registration,"Martello, Robert",[],[],"[23925034, 99052948, 29826367, 91614816, 95699..."
1,AHS CAP SPR 0708FA,AHS CAP SPR,0708FA,AHS CapstoneSpring Pre-registration,"Lynch, Caitrin",[],[],"[30876146, 98014488, 67373835, 67291012, 84935..."
2,AHS1101 0203FA,AHS1101,0203FA,History and Society Bodies in Motion: Migratio...,"Fleischmann, Fritz",[],[],"[39854219, 16495441, 74816072, 85835441, 28902..."
3,AHS1101 0304FA,AHS1101,0304FA,History and Society Bodies in Motion: Migratio...,"Fleischmann, Fritz",[],[],"[42715736, 29826367, 10463379, 22502956, 91614..."
4,AHS1102 0203FA,AHS1102,0203FA,Arts and Humanities The Quest,"Tropp, Martin",[],[],"[41057699, 96482981, 16062888, 59108778, 39329..."


In [5]:
# Checking total number of unique data in each column of data set
print 'total no. of cases: ', len(data)
for field in data:
    if field in ['Id']:
        continue
    else:
        print 'no. of unique data in ' + field + ':', len(data[field].unique())

total no. of cases:  1614
no. of unique data in code-semester: 1614
no. of unique data in code: 306
no. of unique data in semester: 28
no. of unique data in name: 490
no. of unique data in prof: 135
no. of unique data in redundant_codes: 1
no. of unique data in prereqs: 1
no. of unique data in student_ids: 1501


## Functions for Determining Mean and SD for Class ver student years

In [7]:
def edit_course_names_in_dataframe(data):
    """
    Rids of repetitive courses with alphabet in the end
    """
    for i in range(len(data['code'])):
        if data['code'][i][-1].isalpha():
            if data['code'][i][-1] is 'R':
                pass
            else:
                data['code'][i] = data['code'][i][0:-1]
        else:
            pass
    return data['code']

def list_all_courses(data):
    """
    Works for the course-semester.csv dataset
    finds mean number of students and standard deviation for each course
    throughout the years
    """
    course_data = edit_course_names_in_dataframe(data)
    holder = []
    for i in range(len(data)):
        if data['code'][i] not in holder:
            holder += [data['code'][i]]
        else:
            continue
    return holder        

def dict_code_index(data):
    """
    generates dictionary linking each course to index in csv
    """
    data2 = data
    data2['code'] = edit_course_names_in_dataframe(data)
    dict_code_index = {}
    for i in range(len(data2['code'])):
        if data2['code'][i] in dict_code_index:
            dict_code_index[data2['code'][i]] = dict_code_index[data2['code'][i]] + [i]
        else:
            dict_code_index[data2['code'][i]] = [i]
    return dict_code_index

def grab_year(data, orig):
    """
    Grabs mean year of students for each class
    """
    data2 = data.copy()
    orig2 = orig.copy()
    code_index = dict_code_index(data)
    dict_grab_year = {}
    for key in code_index:
        for index in code_index[key]:
            for student in data2['student_ids'][index][1:-1].split(', '):
                hold = orig2[orig2['studentid'] == float(student)]
                hold = orig2[orig2['code-semester'] == data2['code-semester'][index]]
                print hold
                
grab_year(data,orig)
    
def dict_code_mean(data):
    """
    Generates dictionary linking each course to mean num of students
    """
    data2 = data
    code_index = dict_code_index(data2)
    dict_code_mean = {}
    
    for key in code_index:
        a = float(0)
        for index in code_index[key]:
            a += len(data2['student_ids'][index][1:-1].split(', '))
        dict_code_mean[key] = a/len(code_index[key])
    return dict_code_mean

from math import sqrt

def dict_code_sd(data):
    data2 = data
    code_mean = dict_code_mean(data2)
    code_index = dict_code_index(data2)
    
    dict_code_sd = {}
    
    for key in code_index:
        a = float(0)
        for index in code_index[key]:
            a += (len(data2['student_ids'][index][1:-1].split(', ')) - code_mean[key])**2
        dict_code_sd[key] = sqrt(a/len(code_index[key]))
    
    return dict_code_sd

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [26]:
type(orig['studentid'][1])

numpy.int64